In [1]:
import sys
import os

# Adiciona o diretório correto ao path
# Ajuste o caminho conforme sua estrutura real
sys.path.insert(0, os.getcwd())
print(f"✅ Diretório atual adicionado ao path: {os.getcwd()}")
import fuzzy_systems as fs

from fuzzy_systems.learning.mandani_learning import MamdaniLearning
from fuzzy_systems import MamdaniSystem


import numpy as np
import matplotlib.pyplot as plt

# Configurar matplotlib
%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11
plt.rcParams['figure.dpi'] = 100


from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, ConfusionMatrixDisplay
import pandas as pd


✅ Diretório atual adicionado ao path: /Users/1moi6/Desktop/Minicurso Fuzzy/fuzzy_systems
Automatic parallelization (joblib)


KeyboardInterrupt: 

In [ ]:
# Função alvo
def funcao_real(x):
    """f(x) = -2x + 5"""
    return -2 * x + 5

# Gerar dados
n_samples = 100
X_train = np.linspace(0, 1, n_samples).reshape(-1, 1)
y_train = funcao_real(X_train.flatten()) + np.random.normal(0, 0.01, n_samples)

X_test = np.linspace(0, 1, 200).reshape(-1, 1)
y_true = funcao_real(X_test.flatten())

# Visualizar
plt.figure(figsize=(10, 6))
plt.plot(X_test, y_true, 'k-', linewidth=3, label='f(x) = -2x + 5', alpha=0.7)
plt.scatter(X_train, y_train, color='red', s=30, alpha=0.5, label=f'Dados (n={n_samples})')
plt.xlabel('x', fontsize=13)
plt.ylabel('y', fontsize=13)
plt.title('Linear Fit by Mandani FIS', fontsize=14, fontweight='bold')
plt.legend(fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()


In [ ]:
# ==================== 3. CRIAR SISTEMA FUZZY ====================
input_universe = X_train.min(), X_train.max()
diff = y_train.max() - y_train.min()
output_universe = (y_train.min() - 0.1*diff, y_train.max() + 0.1*diff)

fis = MamdaniSystem.create_automatic(n_inputs=1,
                        n_outputs=1,
                        n_mfs=[5,7],
                        input_universes=input_universe,
                        output_universes=output_universe,
                        mf_type='gaussian',
                        )
fis.info()
fis.plot_variables()

In [ ]:
learner = MamdaniLearning(fis, num_points=1000, verbose=True)

In [ ]:
learner.fit_rules(
            X_train, y_train,
            optimizer='sa',
            optimizer_params={'temperature_init': 100.0, 'cooling_rate': 0.95, 'max_iterations': 5000,'temperature_min': 1e-6},
            initial_solution_method='random'
        )

y_pred_train = learner.predict(X_train)
y_pred = learner.predict(X_test)

rmse_train = learner.get_cost()
rmse_test = learner.score(X_test, y_true)

In [ ]:
# Criar visualização
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

history = learner.get_history()

# 1. Aproximação
axes[0, 0].plot(X_test, y_true, 'b-', linewidth=3, label='Real', alpha=0.7)
axes[0, 0].plot(X_test, y_pred, 'r--', linewidth=2.5, label='FIS', alpha=0.8)
axes[0, 0].scatter(X_train, y_train, alpha=0.3, s=20, c='blue')
axes[0, 0].set_xlabel('x', fontsize=12)
axes[0, 0].set_ylabel('y', fontsize=12)
axes[0, 0].set_title(f'Aproximação\nRMSE = {rmse_test:.4f}', 
                     fontsize=13, fontweight='bold')
axes[0, 0].legend(fontsize=10)
axes[0, 0].grid(True, alpha=0.3)

# 2. Erro
error = y_pred - y_true
axes[0, 1].plot(X_test, error, 'g-', linewidth=2.5)
axes[0, 1].axhline(y=0, color='k', linestyle='--')
axes[0, 1].fill_between(X_test.flatten(), 0, error, alpha=0.3, color='green')
axes[0, 1].set_xlabel('x', fontsize=12)
axes[0, 1].set_ylabel('Erro', fontsize=12)
axes[0, 1].grid(True, alpha=0.3)

# 3. Convergência PSO
axes[1, 0].plot(history['current_cost'], linewidth=2.5, color='darkblue')
axes[1, 0].set_xlabel('Iteração', fontsize=12)
axes[1, 0].set_ylabel('Fitness (RMSE)', fontsize=12)
axes[1, 0].set_title('Convergência FIS', fontsize=13, fontweight='bold')
axes[1, 0].grid(True, alpha=0.3)

# 4. Distribuição do Erro
axes[1, 1].hist(error, bins=30, edgecolor='black', alpha=0.7, color='green')
axes[1, 1].axvline(x=0, color='r', linestyle='--', linewidth=2)
axes[1, 1].set_xlabel('Erro', fontsize=12)
axes[1, 1].set_ylabel('Frequência', fontsize=12)
axes[1, 1].set_title('Distribuição do Erro', fontsize=13, fontweight='bold')
axes[1, 1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()


In [ ]:
# Load dataset
iris = load_iris()

X = iris.data # Shape (150, 4) - 4 features
y = iris.target
y_binary = (y != 0).astype(int)  # 0 = Setosa, 1 = Não-Setosa


feature_names = iris.feature_names 
class_names = iris.target_names

X = X[:,2:4]
feature_names = ['Petal Length (cm)', 'Petal Width (cm)']
class_names = ['setosa', 'versicolor', 'virginica']


#Visualize separability
fig, ax = plt.subplots(figsize=(10, 8))

colors = ['red', 'blue']
labels = ['Setosa', 'Non-Setosa']

for class_id in [0, 1]:
    mask = y_binary == class_id
    ax.scatter(X[mask, -2], X[mask, -1], 
              c=colors[class_id], label=labels[class_id],
              s=80, alpha=0.7, edgecolors='black', linewidth=1)

ax.set_xlabel(iris.feature_names[-2] + ' (cm)', fontsize=13)
ax.set_ylabel(iris.feature_names[-1] + ' (cm)', fontsize=13)
ax.set_title('Iris Dataset - 2 Features (Petal)', fontsize=15, weight='bold')
ax.legend(fontsize=12)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

X_train, X_temp, y_train, y_temp = train_test_split(
    X, y_binary, test_size=0.3, random_state=42, stratify=y_binary
)

X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

# Normalize data
scaler = StandardScaler()
X_train_norm = scaler.fit_transform(X_train)
X_val_norm = scaler.transform(X_val)
X_test_norm = scaler.transform(X_test)


for i in range(X_train.shape[1]):
    print(f'Feature {i+1} - Train: [{X_train_norm[:, i].min():.2f}, {X_train_norm[:, i].max():.2f}] | '
          f'Validation: [{X_val_norm[:, i].min():.2f}, {X_val_norm[:, i].max():.2f}] | '
          f'Test: [{X_test_norm[:, i].min():.2f}, {X_test_norm[:, i].max():.2f}]')

In [ ]:
# ==================== 3. CRIAR SISTEMA FUZZY ====================

fis = MamdaniSystem()
num_features = X_train.shape[1]
num_input_mfs = 2
for idx in range(num_features):
    universe = (X_train_norm[:,idx].min(), X_train_norm[:,idx].max())
    fis.add_input(feature_names[idx], universe)
    input_centers = np.linspace(universe[0], universe[1], num_input_mfs)
    del_in = (input_centers[1]-input_centers[0])
    for i in range(num_input_mfs):
        a =  input_centers[i] - del_in
        b =  input_centers[i]
        c =  input_centers[i] + del_in
        fis.add_term(feature_names[idx], f'MF_{i+1}', 'triangular', (a, b, c))


fis.add_output('setosa', (0, 1))
num_output_mfs = 2
fis.add_term('setosa', 'No', 'triangular', (0, 0.0, 1.0))
fis.add_term('setosa', 'Yes', 'triangular', (0.0, 1.0, 1.0))
fis.info()

In [ ]:
learner = MamdaniLearning(fis, num_points=1000, verbose=True)

In [ ]:
learner.fit_rules(
            X_train_norm, y_train,
            optimizer='sa',
            optimizer_params={'temperature_init': 100.0, 'cooling_rate': 0.95, 'max_iterations': 5000,'temperature_min': 1e-6},
            initial_solution_method='random'
        )

y_pred_train = learner.predict(X_train_norm)
y_pred_test = learner.predict(X_test_norm)

rmse_train = learner.get_cost()
rmse_test = learner.score(X_test_norm, y_test)

In [ ]:
y_pred_test_classes = (y_pred_test >= 0.5).astype(int)
cm = confusion_matrix(y_test, y_pred_test_classes)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names[1:])
disp.plot(cmap=plt.cm.Blues)
plt.title('Confusion Matrix - Iris Dataset (Non-Setosa vs Setosa)')
plt.show()

In [ ]:
rule_matrix = fis.plot_rule_matrix()

In [ ]:
history = learner.get_history()
plt.figure(figsize=(10, 6))
plt.plot(history['current_cost'], label='Training Cost', color='blue')
plt.plot(history['best_cost'], label='Best Cost', color='red')
plt.xlabel('Iteration')
plt.ylabel('Cost (RMSE)')
plt.title('Training Cost over Iterations')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
df = fis.rules_to_dataframe()
df

In [ ]:
# Configurações dos otimizadores
optimizers_config = {
    'SA': {
        'optimizer': 'sa',
        'params': {
            'temperature_init': 100.0,
            'cooling_rate': 0.95,
            'max_iterations': 2000,
            'plateau_iterations': 500,
            'temperature_min': 1e-6
        }
    },
    'GA': {
        'optimizer': 'ga',
        'params': {
            'pop_size': 50,
            'max_gen': 100,
            'mutation_rate': 0.1,
            'crossover_rate': 0.8
        }
    },
    'PSO': {
        'optimizer': 'pso',
        'params': {
            'n_particles': 30,
            'n_iterations': 100,
            'w_max': 0.9,
            'w_min': 0.4
        }
    },
    'DE': {
        'optimizer': 'de',
        'params': {
            'pop_size': 30,
            'max_iter': 100,
            'F': 0.8,
            'CR': 0.7
        }
    }
}


In [ ]:
learner = MamdaniLearning(fis, num_points=1000, verbose=True)
optimizer ='GA'
learner.fit_rules(
            X_train_norm, y_train,
            optimizer=optimizers_config[optimizer]['optimizer'],
            optimizer_params=optimizers_config[optimizer]['params'],
            initial_solution_method='random'
        )